In [2]:
import pandas as pd
import itertools
import numpy as np

In [3]:
csv= '/home/mei/nas/docker/thesis/data/csv/'

In [4]:
def round_up(x, base=5):
    return base * round(x/base)

In [5]:
print('==> Loading data from timeseries files...')
timeseries_lab = pd.read_csv(csv + 'timeserieslab.csv', low_memory=False)
timeseries_periodic = pd.read_csv(csv+ 'timeseriesperiodic.csv')

==> Loading data from timeseries files...


In [6]:
print("there are {} patients in the  and {} records in lab table.".format(len(list(timeseries_lab ['patientunitstayid'].unique())),len(timeseries_lab)))
print("there are {} patients in the  and {} records in vital periodic table.".format(len(list(timeseries_periodic ['patientunitstayid'].unique())),len(timeseries_periodic)))

there are 12260 patients in the  and 2337787 records in lab table.
there are 12260 patients in the  and 10671165 records in vital periodic table.


In [7]:
timeseries_lab.set_index(['patientunitstayid','labresultoffset'], inplace=True)
timeseries_periodic.set_index(['patientunitstayid','observationoffset'], inplace=True)

In [8]:
timeseries_lab.rename(round_up, level = 'labresultoffset', inplace = True)
timeseries_periodic.rename(round_up, level = 'observationoffset', inplace = True)


In [9]:
timeseries_lab.sort_index(inplace=True)
timeseries_lab

labname  labresult
patientunitstayid labresultoffset                            
252784            20                          paO2    1143.80
                  20                            pH       7.05
                  20                         paCO2       9.50
                  20                    O2 Sat (%)      99.00
                  20                   Base Excess     -25.60
...                                            ...        ...
3348105           6060             bedside glucose     159.00
                  6340             bedside glucose     170.00
                  6625             bedside glucose     177.00
                  6885             bedside glucose     196.00
                  7540             bedside glucose     162.00

[2337787 rows x 2 columns]

In [10]:
timeseries_periodic.sort_index(inplace=True)
timeseries_periodic

temperature   sao2  heartrate  \
patientunitstayid observationoffset                                  
252784            10                         NaN  100.0      106.0   
                  15                         NaN  100.0      107.0   
                  20                         NaN  100.0      109.0   
                  25                         NaN  100.0      110.0   
                  30                         NaN   98.0      116.0   
...                                          ...    ...        ...   
3348105           4045                       NaN    NaN       85.0   
                  4050                       NaN    NaN       86.0   
                  4055                       NaN    NaN       97.0   
                  4060                       NaN    NaN       97.0   
                  4065                       NaN    NaN      113.0   

                                     respiration  cvp  systemicsystolic  \
patientunitstayid observationoffset                                       
252784            10                        22.0  NaN               NaN   
                  15                        24.0  NaN               NaN   
                  20                        21.0  NaN               NaN   
                  25                        26.0  NaN               NaN   
                  30                        26.0  NaN               NaN   
...                                          ...  ...               ...   
3348105           4045                       NaN  NaN               NaN   
                  4050                       NaN  NaN               NaN   
                  4055                       NaN  NaN               NaN   
                  4060                       NaN  NaN               NaN   
                  4065                       NaN  NaN               NaN   

                                     systemicdiastolic  systemicmean   st1  \
patientunitstayid observationoffset                                          
252784            10                               NaN           NaN  0.05   
                  15                               NaN           NaN  0.10   
                  20                               NaN           NaN  0.00   
                  25                               NaN           NaN  0.10   
                  30                               NaN           NaN  0.00   
...                                                ...           ...   ...   
3348105           4045                             NaN           NaN   NaN   
                  4050                             NaN           NaN   NaN   
                  4055                             NaN           NaN   NaN   
                  4060                             NaN           NaN   NaN   
                  4065                             NaN           NaN   NaN   

                                      st2  st3  
patientunitstayid observationoffset             
252784            10                 0.05  0.0  
                  15                 0.00 -0.1  
                  20                 0.10  0.0  
                  25                 0.00 -0.1  
                  30                 0.00  0.0  
...                                   ...  ...  
3348105           4045                NaN  NaN  
                  4050                NaN  NaN  
                  4055                NaN  NaN  
                  4060                NaN  NaN  
                  4065                NaN  NaN  

[10671165 rows x 11 columns]

In [11]:
def reconfigure_timeseries(timeseries, offset_column, feature_column=None, test=False):
    """
    Reconfigure timeseries data by setting multi-index and pivoting if necessary.
    """
    timeseries.reset_index(inplace=True)
    if test:
        timeseries = timeseries.iloc[:5000]  # Limit for testing
    timeseries.set_index(['patientunitstayid', pd.to_timedelta(timeseries[offset_column], unit='min')], inplace=True)
    timeseries.drop(columns=offset_column, inplace=True)
    if feature_column:
        timeseries = timeseries.pivot_table(columns=feature_column, index=timeseries.index)
    timeseries.index = pd.MultiIndex.from_tuples(timeseries.index, names=['patient', 'time'])
    return timeseries

In [12]:
test=False
print('==> Reconfiguring lab timeseries...')
lab = reconfigure_timeseries(timeseries_lab, 'labresultoffset', 'labname', test)
lab.columns=lab.columns.droplevel()

print('==> Reconfiguring periodic timeseries...')
periodic = reconfigure_timeseries(timeseries_periodic, 'observationoffset', test)


==> Reconfiguring lab timeseries...
==> Reconfiguring periodic timeseries...


In [13]:
lab 

labname                  -bands  -basos  -eos  -lymphs  -monos  -polys  \
patient time                                                             
252784  0 days 00:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 00:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 02:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
...                         ...     ...   ...      ...     ...     ...   
3348105 4 days 05:00:00     NaN     NaN   NaN      NaN     NaN     NaN   
        4 days 09:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        4 days 14:25:00     NaN     NaN   NaN      NaN     NaN     NaN   
        4 days 18:45:00     NaN     NaN   NaN      NaN     NaN     NaN   
        5 days 05:40:00     NaN     NaN   NaN      NaN     NaN     NaN   

labname                  24 h urine protein  24 h urine urea nitrogen  \
patient time                                                            
252784  0 days 00:20:00                 NaN                       NaN   
        0 days 00:40:00                 NaN                       NaN   
        0 days 01:20:00                 NaN                       NaN   
        0 days 01:40:00                 NaN                       NaN   
        0 days 02:35:00                 NaN                       NaN   
...                                     ...                       ...   
3348105 4 days 05:00:00                 NaN                       NaN   
        4 days 09:40:00                 NaN                       NaN   
        4 days 14:25:00                 NaN                       NaN   
        4 days 18:45:00                 NaN                       NaN   
        5 days 05:40:00                 NaN                       NaN   

labname                  ALT (SGPT)  ANF/ANA  ...  total protein  transferrin  \
patient time                                  ...                               
252784  0 days 00:20:00         NaN      NaN  ...            NaN          NaN   
        0 days 00:40:00         NaN      NaN  ...            NaN          NaN   
        0 days 01:20:00         NaN      NaN  ...            NaN          NaN   
        0 days 01:40:00         NaN      NaN  ...            NaN          NaN   
        0 days 02:35:00         NaN      NaN  ...            NaN          NaN   
...                             ...      ...  ...            ...          ...   
3348105 4 days 05:00:00         NaN      NaN  ...            NaN          NaN   
        4 days 09:40:00         NaN      NaN  ...            NaN          NaN   
        4 days 14:25:00         NaN      NaN  ...            NaN          NaN   
        4 days 18:45:00         NaN      NaN  ...            NaN          NaN   
        5 days 05:40:00         NaN      NaN  ...            NaN          NaN   

labname                  triglycerides  troponin - I  troponin - T  uric acid  \
patient time                                                                    
252784  0 days 00:20:00            NaN           NaN           NaN        NaN   
        0 days 00:40:00            NaN           NaN           NaN        NaN   
        0 days 01:20:00            NaN           NaN           NaN        NaN   
        0 days 01:40:00            NaN           NaN           NaN        NaN   
        0 days 02:35:00            NaN           NaN           NaN        NaN   
...                                ...           ...           ...        ...   
3348105 4 days 05:00:00            NaN           NaN           NaN        NaN   
        4 days 09:40:00            NaN           NaN           NaN        NaN   
        4 days 14:25:00            NaN           NaN           NaN        NaN   
        4 days 18:45:00            NaN           NaN           NaN        NaN   
        5 days 05:40:00            NaN           NaN         

In [14]:
min=lab.index.get_level_values('time').min()    
max=lab.index.get_level_values('time').max()
print(f"最小 labresultoffset: {min}")
print(f"最大 labresultoffset: {max}")

最小 labresultoffset: 0 days 00:00:00
最大 labresultoffset: 97 days 21:50:00


In [15]:
lab = lab.reset_index()  
lab['time'] = pd.to_timedelta(lab['time'])

lab = lab[lab['time'] <= pd.to_timedelta('14 days')]

lab = lab.set_index(['patient', 'time'])

In [16]:
flat=pd.read_csv(csv + 'preprocessed_flat_drug.csv')
labels = pd.read_csv(csv + 'preprocessed_labels.csv')
diagnoses= pd.read_csv(csv + 'preprocessed_diagnoses.csv')


In [17]:
common_id = list(set(flat['patient']).intersection(set(labels['patient']).intersection(set(diagnoses['patient']))))

In [18]:
len(common_id)

11698

In [19]:
lab = lab.reset_index()
lab = lab[lab['patient'].isin(common_id)]
lab = lab.set_index(['patient', 'time'])

periodic = periodic.reset_index()
periodic = periodic[periodic['patient'].isin(common_id)]
periodic = periodic.set_index(['patient', 'time'])

In [20]:
print('==> Combining data together...')
merged = pd.concat([lab, periodic], axis=0, sort=False)

==> Combining data together...


In [21]:
possible_value_ranges = {
   "temperature": (32,43),        # Temperature
    "sao2": (40, 100),             # SpO2
    "heartrate": (30, 400),        # Heart rate
    "respiration": (0, 60),        # Resp. rate
    "cvp": (0, 20),                # CVP
    
    # "etco2": (0, 60),             # EtCO2
    "systemicsystolic": (40, 300), # BP systolic
    "systemicdiastolic": (20, 150),# BP diastolic
    "systemicmean": (30, 200),     # BP mean
}

In [22]:
def filter_vital_signs(data, ranges):
    for column, (min_val, max_val) in ranges.items():
        if column in data.columns:
            data = data[(data[column].isna()) | ((data[column] >= min_val) & (data[column] <= max_val))]
    return data

merged = filter_vital_signs(merged, possible_value_ranges)


In [23]:
# 计算 0.1% 和 99.9% 分位数
low_quantile = merged.quantile(0.001, numeric_only=True)  # 0.1% 分位数
high_quantile= merged.quantile(0.999, numeric_only=True)  # 99.9% 分位数

# 仅保留在 [0.1%, 99.9%] 之间的值
merged = merged[(merged >= low_quantile) & (merged <= high_quantile)]

print("select valid vlaue of vital signs")
print("There are {} patients and {} records in the vital periodic table.".format(
    merged.index.get_level_values('patient').nunique(),
    len(merged)
))

select valid vlaue of vital signs
There are 11698 patients and 10237638 records in the vital periodic table.


In [ ]:
# resampled = merged.groupby(level=0).resample('5min', level=1).mean().ffill() # Resample to 5-minute intervals, level 0 is patient, level 1 is time

In [24]:
merged

-bands  -basos  -eos  -lymphs  -monos  -polys  \
patient time                                                             
252784  0 days 00:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 00:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 01:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        0 days 02:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
...                         ...     ...   ...      ...     ...     ...   
3348105 2 days 19:25:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:30:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
        2 days 19:45:00     NaN     NaN   NaN      NaN     NaN     NaN   

                         24 h urine protein  24 h urine urea nitrogen  \
patient time                                                            
252784  0 days 00:20:00                 NaN                       NaN   
        0 days 00:40:00                 NaN                       NaN   
        0 days 01:20:00                 NaN                       NaN   
        0 days 01:40:00                 NaN                       NaN   
        0 days 02:35:00                 NaN                       NaN   
...                                     ...                       ...   
3348105 2 days 19:25:00                 NaN                       NaN   
        2 days 19:30:00                 NaN                       NaN   
        2 days 19:35:00                 NaN                       NaN   
        2 days 19:40:00                 NaN                       NaN   
        2 days 19:45:00                 NaN                       NaN   

                         ALT (SGPT)  ANF/ANA  ...  sao2  heartrate  \
patient time                                  ...                    
252784  0 days 00:20:00         NaN      NaN  ...   NaN        NaN   
        0 days 00:40:00         NaN      NaN  ...   NaN        NaN   
        0 days 01:20:00         NaN      NaN  ...   NaN        NaN   
        0 days 01:40:00         NaN      NaN  ...   NaN        NaN   
        0 days 02:35:00         NaN      NaN  ...   NaN        NaN   
...                             ...      ...  ...   ...        ...   
3348105 2 days 19:25:00         NaN      NaN  ...   NaN       85.0   
        2 days 19:30:00         NaN      NaN  ...   NaN       86.0   
        2 days 19:35:00         NaN      NaN  ...   NaN       97.0   
        2 days 19:40:00         NaN      NaN  ...   NaN       97.0   
        2 days 19:45:00         NaN      NaN  ...   NaN      113.0   

                         respiration  cvp  systemicsystolic  \
patient time                                                  
252784  0 days 00:20:00          NaN  NaN               NaN   
        0 days 00:40:00          NaN  NaN               NaN   
        0 days 01:20:00          NaN  NaN               NaN   
        0 days 01:40:00          NaN  NaN               NaN   
        0 days 02:35:00          NaN  NaN               NaN   
...                              ...  ...               ...   
3348105 2 days 19:25:00          NaN  NaN               NaN   
        2 days 19:30:00          NaN  NaN               NaN   
        2 days 19:35:00          NaN  NaN               NaN   
        2 days 19:40:00          NaN  NaN               NaN   
        2 days 19:45:00          NaN  NaN               NaN   

                         systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                             
252784  0 days 00:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:40:00                NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                NaN           NaN  NaN  NaN  NaN  
        0 d

In [25]:
def gen_patient_chunk(patients, merged, size=500): # 500 is a good size for LSTM
    """
    Generate patient data chunks for processing.
    """
    it = iter(patients)
    chunk = list(itertools.islice(it, size))
    while chunk:
        yield merged.loc[chunk]
        chunk = list(itertools.islice(it, size))

In [58]:
test=False

def resample_and_mask_independent(timeseries, csv, header, mask_decay=True, decay_rate=4/3, test=False, verbose=False):
    """
    Resample timeseries data to 5-minute intervals for each patient independently, maintaining patient_id as an index.
    """
    if verbose:
        print('Resampling to 5-minute intervals...')
    
    # 存储所有 resampled 结果
    resampled_data = []
    
    # 遍历每个患者进行 resample
    for patient_id, group in timeseries.groupby(level=0):
        # 仅保留时间索引
        group = group.droplevel(0)
        group.index = group.index.ceil(freq='5min')  # 向上取整到最近 5 分钟
        resampled = group.resample('5min', closed='right', label='right').mean()

        # 计算 mask
        if mask_decay:
            if verbose:
                print(f'Calculating mask decay features for patient {patient_id}...')
            mask_bool = resampled.notnull()
            mask = mask_bool.astype(int)
            mask.replace({0: np.nan}, inplace=True)
            inv_mask_bool = ~mask_bool
            count_non_measurements = inv_mask_bool.cumsum() - inv_mask_bool.cumsum().where(mask_bool).ffill().fillna(0)
            mask = mask.ffill().fillna(0) / (count_non_measurements * decay_rate).replace(0, 1)
        else:
            mask = resampled.notnull().astype(int)
        
        # 前向填充
        resampled = resampled.ffill()

        # 组合 mask 数据
        mask.columns = [str(col) + '_mask' for col in mask.columns]
        combined = pd.concat([resampled, mask], axis=1)

        # **关键：确保 patient_id 仍作为索引**
        combined["patient_id"] = patient_id
        combined.set_index("patient_id", append=True, inplace=True)
        
        resampled_data.append(combined)

    # **合并所有患者数据，并确保索引结构**
    final_data = pd.concat(resampled_data)
    final_data = final_data.reorder_levels(["patient_id", "time"]).sort_index()

    if verbose:
        print('Saving progress...')
    if test == False:  # 保存 CSV
        final_data.to_csv(csv + 'preprocessed_timeseries.csv', mode='a', header=header)

    return final_data



In [53]:
test=False

def resample_and_mask(timeseries, csv, header, mask_decay=True, decay_rate=4/3, test=False, verbose=False):
    """
    Resample timeseries data to hourly intervals and calculate masking features.
    """
    if verbose:
       print('Resampling to 5-minute intervals...')
    timeseries = timeseries.groupby(level=[0, 1]).mean()
    unstacked = timeseries.unstack(level=0)
    unstacked.index = unstacked.index.ceil(freq='5min') # 向上取整到最近 5 分钟
    resampled = unstacked.resample('5min', closed='right', label='right').mean()

    if mask_decay:
        if verbose:
            print('Calculating mask decay features...')
        mask_bool = resampled.notnull()
        mask = mask_bool.astype(int)
        mask.replace({0: np.nan}, inplace=True)    
        inv_mask_bool = ~mask_bool
        count_non_measurements = inv_mask_bool.cumsum() - inv_mask_bool.cumsum().where(mask_bool).ffill().fillna(0)
        mask = mask.ffill().fillna(0) / (count_non_measurements * decay_rate).replace(0, 1)
    else:
        if verbose:
            print('Calculating binary mask features...')
        mask = resampled.notnull().astype(int)

    if verbose:
        print('Filling missing data forwards...')
    resampled = resampled.ffill()  # 前向填充
    resampled.index = list(range(1, len(resampled) + 1)) # 重新索引
    mask.index = list(range(1, len(mask) + 1))

    if verbose:
        print('Filling in remaining values with zeros...')
    resampled.fillna(0, inplace=True)

    if verbose:
        print('Reconfiguring and combining features with mask features...')
        
    resampled = resampled.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    mask = mask.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    
    mask.columns = [str(col) + '_mask' for col in mask.columns]
    final = pd.concat([resampled, mask], axis=1)

    if verbose:     # if verbose=True, print the progress
        print('Saving progress...')
    if test==False:  # if test=False, save the progress
        final.to_csv(csv + 'preprocessed_timeseries.csv', mode='a', header=header) # Save to CSV,a for append
    return final

In [59]:
## select top 1000 of merged 
testing = merged.iloc[:500]

In [60]:
import gc

patients = testing.index.unique(level=0)
gen_chunks = gen_patient_chunk(patients, testing)
header = True
print('==> Initiating main processing loop...')
for i, patient_chunk in enumerate(gen_chunks, start=1):
    final=resample_and_mask_independent(patient_chunk,csv,header ,mask_decay=True, decay_rate=4/3, test=test, verbose=False)
    print(f'==> Processed {i * 500} patients...')
    header = False
    
    # Clear memory
    del patient_chunk
    gc.collect()

==> Initiating main processing loop...
==> Processed 500 patients...


In [61]:
processed_ts = pd.read_csv(csv + 'preprocessed_timeseries.csv', index_col=[0, 1])

In [62]:
processed_ts

-bands  -basos  -eos  -lymphs  -monos  -polys  \
patient_id time                                                             
252784     0 days 00:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
           0 days 00:25:00     NaN     NaN   NaN      NaN     NaN     NaN   
           0 days 00:30:00     NaN     NaN   NaN      NaN     NaN     NaN   
           0 days 00:35:00     NaN     NaN   NaN      NaN     NaN     NaN   
           0 days 00:40:00     NaN     NaN   NaN      NaN     NaN     NaN   
...                            ...     ...   ...      ...     ...     ...   
317424     1 days 06:10:00     NaN     NaN   NaN      NaN     NaN     NaN   
           1 days 06:15:00     NaN     NaN   NaN      NaN     NaN     NaN   
           1 days 06:20:00     NaN     NaN   NaN      NaN     NaN     NaN   
           1 days 06:25:00     NaN     NaN   NaN      NaN     NaN     NaN   
           1 days 06:30:00     NaN     NaN   NaN      NaN     NaN     NaN   

                            24 h urine protein  24 h urine urea nitrogen  \
patient_id time                                                            
252784     0 days 00:20:00                 NaN                       NaN   
           0 days 00:25:00                 NaN                       NaN   
           0 days 00:30:00                 NaN                       NaN   
           0 days 00:35:00                 NaN                       NaN   
           0 days 00:40:00                 NaN                       NaN   
...                                        ...                       ...   
317424     1 days 06:10:00                 NaN                       NaN   
           1 days 06:15:00                 NaN                       NaN   
           1 days 06:20:00                 NaN                       NaN   
           1 days 06:25:00                 NaN                       NaN   
           1 days 06:30:00                 NaN                       NaN   

                            ALT (SGPT)  ANF/ANA  ...  sao2_mask  \
patient_id time                                  ...              
252784     0 days 00:20:00         NaN      NaN  ...        0.0   
           0 days 00:25:00         NaN      NaN  ...        0.0   
           0 days 00:30:00         NaN      NaN  ...        0.0   
           0 days 00:35:00         NaN      NaN  ...        0.0   
           0 days 00:40:00         NaN      NaN  ...        0.0   
...                                ...      ...  ...        ...   
317424     1 days 06:10:00         NaN      NaN  ...        0.0   
           1 days 06:15:00         NaN      NaN  ...        0.0   
           1 days 06:20:00         NaN      NaN  ...        0.0   
           1 days 06:25:00         NaN      NaN  ...        0.0   
           1 days 06:30:00         NaN      NaN  ...        0.0   

                            heartrate_mask  respiration_mask  cvp_mask  \
patient_id time                                                          
252784     0 days 00:20:00             0.0               0.0       0.0   
           0 days 00:25:00             0.0               0.0       0.0   
           0 days 00:30:00             0.0               0.0       0.0   
           0 days 00:35:00             0.0               0.0       0.0   
           0 days 00:40:00             0.0               0.0       0.0   
...                                    ...               ...       ...   
317424     1 days 06:10:00             0.0               0.0       0.0   
           1 days 06:15:00             0.0               0.0       0.0   
           1 days 06:20:00             0.0               0.0       0.0   
           1 days 06:25:00             0.0               0.0       0.0   
           1 days 06:30:00             0.0               0.0       0.0   

                            systemicsystolic_mask  systemicdiastolic_mask  \
patient_id time                                                             
252784     0 days 00:20:00                    0.

In [63]:
patient_id =  312047

q = processed_ts.loc[patient_id]
q

,-bands,-basos,-eos,-lymphs,-monos,-polys,24 h urine protein,24 h urine urea nitrogen,ALT (SGPT),ANF/ANA,...,sao2_mask,heartrate_mask,respiration_mask,cvp_mask,systemicsystolic_mask,systemicdiastolic_mask,systemicmean_mask,st1_mask,st2_mask,st3_mask
time,,,,,,,,,,,,,,,,,,,,,
0 days 01:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6 days 23:25:00,NaN,NaN,4.0,14.0,8.0,74.0,NaN,NaN,37.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6 days 23:30:00,NaN,NaN,4.0,14.0,8.0,74.0,NaN,NaN,37.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6 days 23:35:00,NaN,NaN,4.0,14.0,8.0,74.0,NaN,NaN,37.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# get select patient
patient_id =  273703

p = processed_ts.loc[patient_id]
p

,-bands,-basos,-eos,-lymphs,-monos,-polys,24 h urine protein,24 h urine urea nitrogen,ALT (SGPT),ANF/ANA,...,sao2_mask,heartrate_mask,respiration_mask,cvp_mask,systemicsystolic_mask,systemicdiastolic_mask,systemicmean_mask,st1_mask,st2_mask,st3_mask
time,,,,,,,,,,,,,,,,,,,,,
0 days 01:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 01:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 10:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 10:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0 days 10:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
patient= processed_ts.index.get_level_values('patient').unique()

In [42]:
patient

Index([252784, 253331, 255112, 258354, 259414, 263567, 264675, 273703, 284434,
       285944, 289102, 291577, 293988, 306940, 311182, 311776, 312047, 312051,
       312331, 313391, 313656, 316935, 317424],
      dtype='int64', name='patient')